# "Power" calculations with Bayesian ($\epsilon, \delta$) - PACMAN

#### By Roberto-Rafael Maura-Rivero and Sreevidya Ayyar

This notebook aims to provide some intuition on perfmorning power calculations if one was to switch from using an RCT to using the ($\epsilon, \delta$) - PACMAN Bandit algorithm for comparing several treatment arms in an experiment. 

First and foremost, recall the relevant definition of power: 

"Suppose our parameter of interest is $\theta$, the average treatment effect of a particular policy. Now, when performing a hypothesis test on this parameter, with specified null and alternative hypotheses (e.g. $H_0: \theta = 0.3$ and $H_1: \theta \ne 0.3$), power is defined as one minus the probability of making a type 2 error."

$\text{Power} = 1- P(\text{fail to reject} H_0|H_A: \theta \ne 0.3) = P(\text{reject} H_0|H_A: \theta \ne 0.3)$

The difficulty of dealing with power is that the classical definition specifies conditioning on an infitite-dimensional set of possible alternative values for $\theta$. When implementing power calculations therefore, it is standard practice to focus on a single alternative value of $\theta$ at a time, i.e.:

$\text{Power} = 1- P(\text{fail to reject} H_0|H_A: \theta = \gamma) = P(\text{reject} H_0|H_A: \theta = \gamma)$

for $\gamma \in \{0.1,.02,0.4,0.5\}$. How to formulate a set of alternatives is generally up to the empirical researcher. One way could be to list values of $\theta$ that the researcher would like to rule out. Another could be to rule out minimum detestable effects drawn from another, comparable experiment.

In our paper, we propose the use of a Bayesian multi-armed bandit algorithm as a way to pin down treatment focus for an RCT. Our algorithm proposes to roll-out multiple treatments simultaneously over sequential experiments, to identify the set of treatment arms that have the highest average treatment effect (with high probability). The intuition is as follows. Instead of conducting a possibly under-powered RCT with 6 treatment arms, we propose that experimenters conduct a pilot study of sequential experiments to identify the set of best-performing arms. Our multi-armed bandit algorithm can be used to conduct a small number of sequential experiments, each requiring much smaller sample sizes than a typical RCT, to identify say the treatment arm(s) with the highest average treatment effect which should be the focus of a larger-scale RCT.

Our algorithm works as follows. In the first round of experimentation, we randomize 8 treatments. Our algorithm assesses the performance of each treatment and does two things. Firstly, it discards the 4 worst-performing treatment. This means in the next round, we will only randomize 4 treatments. Secondly, our algorithm uses the information gathered on remaining 4 best-performing treatments to decide how many people should be allocated to each treatment in the next round of the experiment. This is the "Bayesian" nature of our algorithm. From here, the algorithm enters into the second round of experimentation, where 4 treatments are randomized. Two are then discarded, and the third round of experimentation is designed only for the 2 remaining treatments. Based on the performance of these 2 treatments in this third round, the algorithm identified the best treatment arm.


If we are using this Bayesian multi-armed bandit algorithm though, what is the relevant notion of power? We believe it is not immediately obvious which object to care about, but here is one interpretation we have. Suppose we have 8 treatment arms; that is, 7 treatments and 1 control. 

Suppose we want to test a very specific alternative hypothesis on the ATE of multiple treatments, e.g. 

$H: \\ \theta_0 = 0, \\ \theta_1 = 0.1,\\ \theta_2 = 0.2,\\ \theta_3 = 0.3,\\ \theta_4 = 0.4,\\ \theta_5 = 0.5,\\ \theta_6 = 0.6,\\ \theta_7 = 0.7$ 

Now, we can calculate the probability of "type 2 error" as the probability of returning the control as the best treatment.

Bandit Power = $Pr$(($\epsilon, \delta$)-PACMAN algorithm proposes the control as best treatment | $H$)

In [10]:
# install + import necessary libraries

!pip install numpy
!pip install tqdm

import numpy as np
from tqdm import trange

In [11]:
# necessary functions

# sample complexity:
def sample_complexity(epsilon_L, delta_L, alpha, beta, n_simulations=1000, n_samples=1000):
    
    n_treatments = len(alpha)
    max_n = 100
    left = 1
    right = max_n
    
    while left <= right:
        n = (left + right) // 2
        simulation_with_success = 0
        for _ in range(n_simulations):
            
            thetas = np.random.beta(alpha, beta, size=n_treatments) 
            highest_theta = np.max(thetas)
            is_epsilon_optimal = np.abs(thetas - highest_theta) <= epsilon_L

            sum_Y = np.random.binomial(n, thetas)
            updated_alpha = alpha + sum_Y
            updated_beta = beta + n - sum_Y

            bool_survived_treatments = bool_choice_rule(updated_alpha, updated_beta, epsilon_L, n_samples)

            success = is_epsilon_optimal * bool_survived_treatments
            if np.sum(success) > 0:
                simulation_with_success += 1

        percentage_success = simulation_with_success / n_simulations

        if percentage_success > 1 - delta_L:
            right = n - 1
        else:
            left = n + 1
    return left

def bool_choice_rule(alpha, beta, epsilon_L, n_samples):
    
    n_treatments = len(alpha)
    
    thetas = np.random.beta(alpha[:, np.newaxis], beta[:, np.newaxis], size=(n_treatments, n_samples))
    highest_theta = np.max(thetas, axis=0)
    is_epsilon_optimal = np.abs(thetas - highest_theta) <= epsilon_L
    pr_epsilon_optimal = np.sum(is_epsilon_optimal, axis=1) / n_samples

    sorted_indices = np.argsort(pr_epsilon_optimal)[::-1]
    top_half_indices = sorted_indices[:n_treatments // 2]
    bool_survived_treatments = np.zeros(n_treatments)
    bool_survived_treatments[top_half_indices] = 1

    return bool_survived_treatments

In [62]:
# researcher-chosen parameters for the algorithm

epsilon = 0.05
epsilon_L = epsilon/3
delta = 0.05
delta_L = delta/3

number_treatments = 8

# hypothesize the average outcome  of each treatment i.e. E(Y|T_i)
# set this manually according to how many treatments you wish to test
# note - these could also be "random" - i.e. drawn from your prior distribution
theta = np.zeros(number_treatments)
theta[0] = 0.50 # this is the control group average outcome
theta[1] = 0.55
theta[2] = 0.55
theta[3] = 0.55
theta[4] = 0.60
theta[5] = 0.60
theta[6] = 0.60
theta[7] = 0.65
print ("the ATE of your treatments are: ")
for i in np.arange(1,number_treatments): # ATE of treatments 1-7 are defined relative to the control group average outcome
    print ("treatment " + str(i) + " : " + str(round(theta[i]-theta[0], 2))) 

the ATE of your treatments are: 
treatment 1 : 0.05
treatment 2 : 0.05
treatment 3 : 0.05
treatment 4 : 0.1
treatment 5 : 0.1
treatment 6 : 0.1
treatment 7 : 0.15


In [63]:
# set prior

# default: prior a uniform distribution for all treatments, i.e. distribution B(1,1)
# same prior for each of the eight treatment arms (7 treatments + 1 control)
def agnostic_prior():
    alphas = np.ones(number_treatments)
    betas = np.ones(number_treatments)
    return alphas, betas

In [67]:
# simulations!
def simulation(theta, prior):
    n_sim = 30  # amount of people used in this simulation
    n_remaining_treatments = number_treatments # begin with full set of treatment arms

    counter_type_2_error = 0
    history_complexity = np.zeros(n_sim)

    for i in trange(n_sim):
        complexity_simulation = 0
        alphas, betas = prior()
        remaining_treatments = np.arange(number_treatments)
        for village in range(3):
            # data recollection
            # number of people allocated to each treatment
            n_treatment = sample_complexity(epsilon_L=epsilon_L,
                        delta_L=delta_L,
                        alpha=alphas,
                        beta=betas,
                        n_simulations=100)
            complexity_simulation += n_treatment
            
            # sample each treatment n_treatment times from a binomial
            # distribution with probability theta_i
            # for i in range(8):
            #     outcome_treatment_i = np.random.binomial(n_treatment, theta[i])
            outcome_treatment = np.random.binomial(n_treatment, theta[remaining_treatments])

            # update prior (now this is posterior distribution from where each ATE is drawn)
            # for i in range(8):
            #     alphas[i] += outcome_treatment_i
            #     betas[i] += n_treatment - outcome_treatment_i
            alphas[remaining_treatments] = alphas[remaining_treatments] + outcome_treatment
            betas[remaining_treatments] = betas[remaining_treatments] + n_treatment - outcome_treatment

            # choose which treatments survive
            # to do that, we first calculate the probability of being epsilon optimal
            
            # SREE: i do not see this step? how are we updating remaining_treatments?
            # Roberto: My bad, idk what happened there. I added it back in now.

            bool_survived_treatments = bool_choice_rule(alphas[remaining_treatments], 
                                                        betas[remaining_treatments], 
                                                        epsilon_L, 
                                                        n_samples=100)
            remaining_treatments = remaining_treatments[bool_survived_treatments == 1]

            # update remaining treatments
            n_remaining_treatments = len(remaining_treatments)
            if int(n_remaining_treatments) == 1:
                # if there is only one treatment left, it is the best one
                # and we can stop the simulation
                break

        history_complexity[i] = complexity_simulation

        if 0 in remaining_treatments: 
            print(remaining_treatments)
            counter_type_2_error += 1
    return history_complexity, counter_type_2_error

history_complexity, counter_type_2_error = simulation(theta, prior)

 60%|██████    | 18/30 [00:25<00:16,  1.40s/it]

[0]


100%|██████████| 30/30 [00:41<00:00,  1.38s/it]


In [65]:
print("epsilon is: " + str(epsilon))
print("delta is: " + str(delta))

# tell me the average complexity of the simulations and relevant quantiles
print("the average complexity of the simulations is: " + str(np.mean(history_complexity)))
print("the 10th percentile of the complexity is: " + str(np.percentile(history_complexity, 10)))
print("the 90th percentile of the complexity is: " + str(np.percentile(history_complexity, 90)))

print ("the probability of type 2 error is: " + str(counter_type_2_error / n_sim))
print ("The \"bandit power\" is " + str(1 - counter_type_2_error / n_sim))

epsilon is: 0.05
delta is: 0.05
the average complexity of the simulations is: 45.5
the 10th percentile of the complexity is: 22.400000000000002
the 90th percentile of the complexity is: 83.0
the probability of type 2 error is: 0.06666666666666667
The "bandit power" is 0.9333333333333333


After playing around with this simulation, you might find some particular values where the power of the bandit algorithm is quite bad (a high number), while the epsilon-delta are both still very small values. How is this possible?


The key here is that the ($\epsilon, \delta$) depend on your prior beliefs about the treatment effects. In this notebook, our default is to assume a uniform prior, but you can change it to whatever you want. 

The uniform prior is uninformative, and so indicates that within a given support, the researcher has no information on what the treatment effect is likely to be. If however you believe (for example) that your treatment effects will be fairly small, and you set the alternative hypothesis to be: 

$H: \\ \theta_0 = 0, \\ \theta_1 = 0.1,\\ \theta_2 = 0.2,\\ \theta_3 = 0.3,\\ \theta_4 = 0.4,\\ \theta_5 = 0.5,\\ \theta_6 = 0.6,\\ \theta_7 = 0.7$ 

then you should reflect that by formulating a prior that puts more density around 0-0.7, and less outside this region. Including such information in your prior will have a lot of effect on the algorithm's performance!

In [66]:
extreme_theta = np.zeros(number_treatments)
extreme_theta[0] = 0.5 # this is the control group average outcome
extreme_theta[1] = 0.5
extreme_theta[2] = 0.5
extreme_theta[3] = 0.5
extreme_theta[4] = 0.5
extreme_theta[5] = 0.5
extreme_theta[6] = 0.5
extreme_theta[7] = 0.551

# technically ATE > epsilon, yet, we are going to have problems with power because of agnostic prior

history_complexity, counter_type_2_error = simulation(extreme_theta, agnostic_prior)

# TODO: define reasonable prior. Plot agnostic prior and reasonable prior 
# explain the intuition that alpha/(alpha+beta) is like the probability of success
# a reasonable prior would have a high alpha and high beta for all treatments and 
# same alpha/(alpha + beta) for all treatments

def reasonable_prior():
    raise NotImplementedError

history_complexity, counter_type_2_error = simulation(extreme_theta, reasonable_prior)


NameError: name 'simulation' is not defined